### Import libraries

In [ ]:
import os
from os import path
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np

# Doc2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# Sklearn
from sklearn import utils
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Tensorflow, Keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

# For multicore
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn.externals.joblib import parallel_backend

tf.random.set_seed(42)
np.random.seed(42)


### Load text transcripts

In [ ]:
def file_preprocess():
  transcribed_dir = "C:/Users/j/Desktop/ADReSSo21/transcribed-correction/"

  comb_df = pd.DataFrame(columns=['file', 'text', 'label'])

  for folder in sorted(os.listdir(transcribed_dir)):
    for file in sorted(os.listdir(transcribed_dir+folder)):

      df = pd.read_csv(transcribed_dir+folder+'/'+file)
      df2 = df[['speaker', 'transcript_correction']]

      # Drop INV rows
      drop_inv = df2[df2['speaker']=='INV'].index
      df3 = df2.drop(drop_inv)
      df3 = df2

      # Combining rows
      lst = []
      for i, row in df3.iterrows():
        lst.append(row['transcript_correction'])

      lst2 = [x for x in lst if pd.isnull(x) == False]

      lst3 = ''.join(lst2)

      dic = {}
      dic['file'] = file
      dic['text'] = lst3
      dic['label'] = folder
      
      comb_df = comb_df.append(dic, ignore_index=True)

      comb_df['label'] = comb_df['label'].replace(['ad'], 1)
      comb_df['label'] = comb_df['label'].replace(['cn'], 0)

  return comb_df

In [ ]:
comb_df = file_preprocess()

### Data Preprocessing

In [ ]:
# Lowercase all text

def clean(text):   
  text = text.lower()
  return text

In [ ]:
# Function to apply clean

def data_preprocess():
  train_df = comb_df.reindex(np.random.permutation(comb_df.index))
  train_df = train_df.drop(columns=['file'])
  train_df['text'] = train_df['text'].apply(clean)
  train = train_df 

  return train

In [ ]:
# Split to text/labels

train = data_preprocess()
X_train, y_train = train['text'], train['label']

### Doc2Vec

#### Machine learning classifierss

##### D2V

In [ ]:
class Doc2VecModel(BaseEstimator):

  def __init__(self, dm=1, size=1, window=1):
    self.d2v_model = None
    # If you are using a later version of gensim you need to change size to vector_size
    self.size = 512
    self.window = window
    self.dm = dm
    self.dbow_words = 1

  def fit(self, raw_documents, y=None):

    # Initialize model
    self.d2v_model = Doc2Vec(size=self.size, window=self.window, dm=self.dm, dbow_words=1, min_count=1, negative=5, hs=0)

    # Tag docs
    tagged_documents = []
    for index, row in raw_documents.iteritems():
      tag = '{}_{}'.format("type", index)
      tokens = row.split()
      tagged_documents.append(TaggedDocument(words=tokens, tags=[tag]))

    # Build vocabulary
    self.d2v_model.build_vocab(tagged_documents)

    # Train model
    self.d2v_model.train(tagged_documents, total_examples=len(tagged_documents), epochs=100)
    return self

  def transform(self, raw_documents):
    X = []
    for index, row in raw_documents.iteritems():
      X.append(self.d2v_model.infer_vector(row))
    X = pd.DataFrame(X, index=raw_documents.index)
    return X

  def fit_transform(self, raw_documents, y=None):
    self.fit(raw_documents)
    return self.transform(raw_documents)



##### Decision Tree

In [ ]:
def decision_tree(X_train, y_train):
  
  param_grid = [{'doc2vec__window': [8],
              'doc2vec__dm': [0]},

              {'classifier': [DecisionTreeClassifier()],
              'classifier__max_depth': np.arange(1,11)}
  ]

  pipe_log = Pipeline([('doc2vec', Doc2VecModel()), 
                     ('classifier', DecisionTreeClassifier()),
                      ]
                    )

  # Grid Search
  grid = GridSearchCV(pipe_log, 
                      param_grid=param_grid,
                      scoring="accuracy",
                      n_jobs=2,
                      cv=10,
                      refit=True)

  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    print('Best Params:', grid.best_params_)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

##### Decision Tree Bagger

In [ ]:
def decision_tree_bagger(X_train, y_train):

  param_grid = [{'doc2vec__window': [8],
            'doc2vec__dm': [0]},

            {'classifier': [BaggingClassifier(DecisionTreeClassifier())],
            'classifier__base_estimator__max_depth': np.arange(1,11)}
  ]

  pipe_log = Pipeline([('doc2vec', Doc2VecModel()), 
                     ('classifier', BaggingClassifier(DecisionTreeClassifier())),
                      ]
                    )

  # Grid Search
  grid = GridSearchCV(pipe_log, 
                      param_grid=param_grid,
                      scoring="accuracy",
                      n_jobs=2,
                      cv=10,
                      refit=True)

  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    print('Best Params:', grid.best_params_)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

##### Random Forest

In [ ]:
def random_forest(X_train, y_train):

  param_grid = [{'doc2vec__window': [8],
            'doc2vec__dm': [0]},

            {'classifier': [RandomForestClassifier()],
            'classifier__n_estimators': [50],
            'classifier__max_depth' : np.arange(1,11)}
  ]

  pipe_log = Pipeline([
                      ('doc2vec', Doc2VecModel()), 
                      ('classifier', RandomForestClassifier())
                      ])

  # Grid Search
  grid = GridSearchCV(pipe_log, 
                      param_grid=param_grid,
                      scoring="accuracy",
                      n_jobs=2,
                      cv=10,
                      refit=True)

  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    print('Best Params:', grid.best_params_)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

##### Random Forest Bagger

In [ ]:
def random_forest_bagger(X_train, y_train):
  
  param_grid = [{'doc2vec__window': [8],
            'doc2vec__dm': [0]},

            {'classifier': [BaggingClassifier(RandomForestClassifier())],
            'classifier__base_estimator__max_depth' : np.arange(1,11),
            'classifier__base_estimator__n_estimators' : [50]}
  ]
  
  pipe_log = Pipeline([
                       ('doc2vec', Doc2VecModel()), 
                      ('classifier', BaggingClassifier(RandomForestClassifier()))
                      ])

  # Grid Search
  grid = GridSearchCV(pipe_log, 
                      param_grid=param_grid,
                      scoring="accuracy",
                      n_jobs=2,
                      cv=10,
                      refit=True)

  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    print('Best Params:', grid.best_params_)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

##### Support Vector

In [ ]:
def support_vector(X_train, y_train):

  param_grid = [{'doc2vec__window': [8],
            'doc2vec__dm': [0]},

            {'classifier': [SVC(max_iter=10000)],
            'classifier__C': np.arange(0.5, 1.6, 0.1),
            'classifier__kernel': ['linear', 'rbf']}
  ]

  pipe_log = Pipeline([
                       ('doc2vec', Doc2VecModel()), 
                     ('classifier', SVC(max_iter=10000))
                      ])

  # Grid Search
  grid = GridSearchCV(pipe_log, 
                      param_grid=param_grid,
                      scoring="accuracy",
                      n_jobs=2,
                      cv=10,
                      refit=True)

  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    print('Best Params:', grid.best_params_)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

##### Log Regression

In [ ]:
def log_reg(X_train, y_train):

  param_grid = [{'doc2vec__window': [8],
            'doc2vec__dm': [0]},

              {'classifier':[LogisticRegression(max_iter=10000)],
              'classifier__C': np.arange(0.5, 1.6, 0.1)},
  ]

  pipe_log = Pipeline([
                       ('doc2vec', Doc2VecModel()), 
                     ('classifier', LogisticRegression(max_iter=10000))
                      ])

  # Grid Search
  grid = GridSearchCV(pipe_log, 
                      param_grid=param_grid,
                      scoring="accuracy",
                      n_jobs=2,
                      cv=10,
                      refit=True)

  with parallel_backend('threading'):

    grid.fit(X_train, y_train)
    print('Best Params:', grid.best_params_)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

##### Training and tuning on 10 fold cross validation

In [ ]:

print('Decision Tree')
decision_tree(X_train, y_train)

print('Decision Tree Bagger')
decision_tree_bagger(X_train, y_train)

print('Random Forest')
random_forest(X_train, y_train)

print('Random Forest Bagger')
random_forest_bagger(X_train, y_train)

print('SVM')
support_vector(X_train, y_train)

print('Logistic Regression')
log_reg(X_train, y_train)

Decision Tree
Best Params: {'classifier': DecisionTreeClassifier(max_depth=7), 'classifier__max_depth': 7}
Mean Test Score: 0.6147058823529411
Std Test Score: 0.15006846764959972
Decision Tree Bagger
Best Params: {'classifier': BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5)), 'classifier__base_estimator__max_depth': 5}
Mean Test Score: 0.6448529411764706
Std Test Score: 0.0915315665858555
Random Forest
Best Params: {'classifier': RandomForestClassifier(max_depth=7, n_estimators=50), 'classifier__max_depth': 7, 'classifier__n_estimators': 50}
Mean Test Score: 0.65
Std Test Score: 0.12422771812907572
Random Forest Bagger
Best Params: {'classifier': BaggingClassifier(base_estimator=RandomForestClassifier(max_depth=9,
                                                        n_estimators=50)), 'classifier__base_estimator__max_depth': 9, 'classifier__base_estimator__n_estimators': 50}
Mean Test Score: 0.64375
Std Test Score: 0.10805884454353121
SVM
Best Params: {'classif

#### Neural Network

##### Setting up Doc2Vec

In [ ]:
# Tag the documents

train_tagged = train.apply(
    lambda r: TaggedDocument(words=(r['text']), tags=[r.label]), axis=1)

In [ ]:
# Setup parameters for Doc2Vec

VECSIZE = 512 #
WINDOWSIZE = 8 #
MINCOUNT = 0
NEGATIVE = 5
HS = 0
DBOW = 1
SETTING = 0 # DBOW/DM

EPOCHS = 100

model = Doc2Vec(dm=SETTING,
            vector_size=VECSIZE,
            window=WINDOWSIZE,
            min_count=MINCOUNT,
            negative=NEGATIVE,
            dbow_words=DBOW,
            hs=HS,
            workers=8)

# Building the vocab
model.build_vocab([x for x in tqdm(train_tagged.values)])

# Training model
model.train(train_tagged, total_examples=len(train_tagged.values), epochs=EPOCHS)

100%|██████████| 166/166 [00:00<?, ?it/s]


In [ ]:
# For inference

def vec_for_learning(model, tagged_docs):
  sents = tagged_docs.values
  targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
  return targets, regressors

In [ ]:
y2_train, X2_train = vec_for_learning(model, train_tagged)
X2_train = np.array(X2_train)
y2_train = np.array(y2_train)

In [ ]:
# Normalize data

scaler = StandardScaler()
X2_train = scaler.fit_transform(X2_train)

In [ ]:
def create_model(neuron, leaky_alpha, dropout_rate, learning_rate):

  print ('Creating model...')

  model = Sequential()

  # Embeddings have dimension of 512 
  model.add(Dense(neuron, input_dim=512))
  model.add(LeakyReLU(alpha=leaky_alpha))
  model.add(Dropout(dropout_rate))

  model.add(Dense(neuron))
  model.add(LeakyReLU(alpha=leaky_alpha))
  model.add(Dropout(dropout_rate))

  model.add(Dense(neuron))
  model.add(LeakyReLU(alpha=leaky_alpha))

  model.add(Dense(1, activation='sigmoid'))

  print ('Compiling...')
  optimizer = Adam(learning_rate=learning_rate)
  
  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'],
                )
  
  return model

In [ ]:
def param_tuning():
      
  # For debugging    
  # param_grid = [
  
  #         { 
  #           'batch_size':[4], 
  #           'epochs':[50],
  #           'leaky_alpha':[0.2],
  #           'dropout_rate':[0.3],
  #           'learning_rate':[1e-3],
  #           'neuron':[32]
  #           }
  # ]

  param_grid = [
    
            { 
              'batch_size':[1,2,4], 
              'epochs':[50],
              'leaky_alpha':[0.2],
              'dropout_rate':[0.3,0.5],
              'learning_rate':[1e-4],
              'neuron':[16, 32]
             }
  ]

  # Grid Search
  grid = GridSearchCV(KerasClassifier(build_fn=create_model), 
                    param_grid=param_grid, 
                    cv=10,
                    refit=True, 
                    n_jobs=2, 
                    scoring="accuracy")
  
  with parallel_backend('threading'):

    grid.fit(X2_train, y2_train)
    MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
    STD = (grid.cv_results_['std_test_score'][grid.best_index_])
    print('Mean Test Score:', MS)
    print('Std Test Score:', STD) 

    best_model = grid.best_estimator_

  return best_model


In [ ]:
best_model = param_tuning()

Creating model...
Creating model...
Compiling...
Compiling...
Epoch 1/50
Epoch 1/50
38/38 [==============================] - 1s 9ms/step - loss: 0.6959 - accuracy: 0.4564
Epoch 2/50
38/38 [==============================] - 2s 13ms/step - loss: 0.6803 - accuracy: 0.6174
Epoch 3/50
Epoch 2/50
38/38 [==============================] - 0s 13ms/step - loss: 0.6418 - accuracy: 0.7047
Epoch 4/50
Epoch 3/50
38/38 [==============================] - 1s 14ms/step - loss: 0.6021 - accuracy: 0.7114
Epoch 5/50
Epoch 4/50
38/38 [==============================] - 1s 14ms/step - loss: 0.6389 - accuracy: 0.7047
Epoch 6/50
Epoch 5/50
38/38 [==============================] - 0s 13ms/step - loss: 0.5927 - accuracy: 0.6913
Epoch 7/50
Epoch 6/50
38/38 [==============================] - 0s 13ms/step - loss: 0.5776 - accuracy: 0.6980
Epoch 8/50
Epoch 7/50
38/38 [==============================] - 0s 12ms/step - loss: 0.6019 - accuracy: 0.7181
Epoch 9/50
Epoch 8/50
38/38 [==============================] - 0s 12ms

### BERT

#### Install dependencies

In [ ]:
!pip install transformers -U

In [ ]:
# Bert
from transformers import BertTokenizerFast, BertModel
import torch

#### Load BERT

In [ ]:
# Load BERT models first

# Use BERT base uncased, set output hidden states to true to extract embeddings
text_model_torch = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# Set max length to 512 and padding and truncate to true
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', truncation=True, padding=True, max_length=512)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Extract embeddings

In [ ]:
def extract_embeddings(text):

  # Special tokens CLS and SEP
  # To mark as classification task
  # To mark end of sentence or separation between two sentences
  marked_text = "[CLS] " + text + " [SEP]"

  # Tokenize sentences with BERT
  tokenized_text = tokenizer.tokenize(marked_text) #, truncation=True, max_length=512)
  # Map tokens to vocab indices
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)

  # Convert to pytorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Set it to eval mode
  text_model_torch.eval()
  
  with torch.no_grad():
    outputs = text_model_torch(tokens_tensor, segments_tensors)
    # Get hidden states from all the layers
    hidden_states = outputs[2]

  # # For word embeddings, if we want to extract the last 4 layers
  # # To combine and reshape the tensors 
  # token_embeddings = torch.stack(hidden_states, dim=0)
  # token_embeddings = torch.squeeze(token_embeddings, dim=1)
  # token_embeddings = token_embeddings.permute(1,0,2)

  # # If we want to concat the last 4 layers
  # token_vecs_cat = []
  # for token in token_embeddings:
  # # Alternatively we can also sum() 
  #   cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
  #   token_vecs_cat.append(cat_vec)

  # For sentence embeddings
  # We take the second-to-last hidden layer and average it 
  token_vecs = hidden_states[-2][0]

  # Calculate the average 
  sentence_embedding = torch.mean(token_vecs, dim=0)

  return sentence_embedding

In [ ]:
# Extract bert embeddings

# The text transcripts generated does not have defined "sentences" 
# Instead the transcripts have staggered full stops, so it's hard to tell which are 'real' sentences. 
# Generated transcript: Transcript 1: ['I saw the girl. stealing the. cookie that was a bad move.']
# Expected transcript: Transcript 1: ['I saw the girl stealing the cookie. That was a bad move.']

X_train_embedd = []
all_sentences = X_train

for sent in all_sentences:

  embedd = extract_embeddings(sent)
  embedd = embedd.cpu().detach().numpy()
  X_train_embedd.append(embedd)


#### Normalize

In [ ]:
scaler = StandardScaler()
X_train_embedd = scaler.fit_transform(X_train_embedd)

#### Setup for training

In [ ]:
def create_model(neuron, leaky_alpha, dropout_rate, learning_rate):
    
  print ('Creating model...')

  model = Sequential()

  model.add(Dense(neuron, input_dim=768))
  model.add(LeakyReLU(alpha=leaky_alpha))
  model.add(Dropout(dropout_rate))

  model.add(Dense(neuron))
  model.add(LeakyReLU(alpha=leaky_alpha))
  model.add(Dropout(dropout_rate))

  model.add(Dense(neuron))
  model.add(LeakyReLU(alpha=leaky_alpha))

  model.add(Dense(1, activation='sigmoid'))

  print ('Compiling...')
  optimizer = Adam(learning_rate=learning_rate)

  
  model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'],
                )
  
  return model

#### Train and tune model

In [ ]:

params = [
          { 
            'batch_size':[1,2,4], 
            'epochs':[50],
            'leaky_alpha':[0.2],
            'dropout_rate':[0.3,0.5],
            'learning_rate':[1e-4],
            'neuron':[16,32]
           }
]

# Grid Search
grid = GridSearchCV(KerasClassifier(build_fn=create_model), 
                  params, 
                  refit=True, 
                  n_jobs=2, 
                  cv=10,
                  scoring="accuracy")

# For multi process
with parallel_backend('threading'):

  grid.fit(X_train_embedd, y_train)
MS = (grid.cv_results_['mean_test_score'][grid.best_index_])
STD = (grid.cv_results_['std_test_score'][grid.best_index_])
print('Mean Test Score:', MS)
print('Std Test Score:', STD) 
# print(grid.best_params_)
best_model = grid.best_estimator_


Creating model...Creating model...

149/149 [==============================] - 1s 6ms/step - loss: 0.2625 - accuracy: 0.8926
Epoch 29/50
149/149 [==============================] - 1s 8ms/step - loss: 0.2308 - accuracy: 0.9060
Epoch 30/50
75/75 [==============================] - 2s 14ms/step - loss: 1.2604 - accuracy: 0.5235
Epoch 2/50
75/75 [==============================] - 2s 13ms/step - loss: 0.8949 - accuracy: 0.5168
Epoch 2/50
149/149 [==============================] - 1s 10ms/step - loss: 0.2580 - accuracy: 0.8993
Epoch 31/50
75/75 [==============================] - 1s 13ms/step - loss: 1.0481 - accuracy: 0.5369
Epoch 3/50
75/75 [==============================] - 1s 14ms/step - loss: 0.9647 - accuracy: 0.5034
Epoch 3/50
75/75 [==============================] - 1s 13ms/step - loss: 0.8644 - accuracy: 0.5705
Epoch 4/50
75/75 [==============================] - 1s 13ms/step - loss: 0.8136 - accuracy: 0.5570
Epoch 4/50
149/149 [==============================] - 1s 10ms/step - loss: 0.

In [ ]:
# Train on best parameters
# params = [
#       { 
#         'batch_size':[2], 
#         'epochs':[50],
#         'leaky_alpha':[0.2],
#         'dropout_rate':[0.5],
#         'learning_rate':[1e-4],
#         'neuron':[32]
#             }
# ]

final_model = create_model(32, 0.2, 0.5, 1e-4)

final_model.fit(X_train_embedd, y_train, epochs=50, batch_size=2, validation_split=0.2, shuffle=True)

Creating model...
Compiling...
Epoch 1/50
5/5 [==============================] - 1s 46ms/step - loss: 1.2890 - accuracy: 0.4242 - val_loss: 0.6608 - val_accuracy: 0.5882
Epoch 2/50
5/5 [==============================] - 0s 11ms/step - loss: 1.1022 - accuracy: 0.4242 - val_loss: 0.6574 - val_accuracy: 0.5882
Epoch 3/50
5/5 [==============================] - 0s 11ms/step - loss: 1.2485 - accuracy: 0.4394 - val_loss: 0.6553 - val_accuracy: 0.5882
Epoch 4/50
5/5 [==============================] - 0s 11ms/step - loss: 1.3139 - accuracy: 0.4621 - val_loss: 0.6539 - val_accuracy: 0.5882
Epoch 5/50
5/5 [==============================] - 0s 13ms/step - loss: 1.1329 - accuracy: 0.4697 - val_loss: 0.6533 - val_accuracy: 0.6176
Epoch 6/50
5/5 [==============================] - 0s 11ms/step - loss: 1.0866 - accuracy: 0.4621 - val_loss: 0.6523 - val_accuracy: 0.6176
Epoch 7/50
5/5 [==============================] - 0s 11ms/step - loss: 1.0158 - accuracy: 0.4621 - val_loss: 0.6519 - val_accuracy: 0.6

In [ ]:
# Save the final model

final_model.save('C:/Users/j/Desktop/ADReSSo21/text_model2')

INFO:tensorflow:Assets written to: C:/Users/j/Desktop/ADReSSo21/text_model2\assets
